<a href="https://colab.research.google.com/github/b0r1ngx/YelpReviewsTonalAnalysingAWD_LSTM/blob/master/Yelp_Reviews_Tonal_Analysing_AWD_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from fastai.text.all import *
# torch.cuda.set_device('cuda:0')
default_device()

device(type='cuda', index=0)

Pick T4 GPU

Download Yelp Reviews

In [ ]:
path = untar_data(URLs.YELP_REVIEWS)

Show what data we have

In [ ]:
path.ls()

(#3) [Path('/root/.fastai/data/yelp_review_full_csv/train.csv'),Path('/root/.fastai/data/yelp_review_full_csv/test.csv'),Path('/root/.fastai/data/yelp_review_full_csv/readme.txt')]

In [ ]:
train_path = 'train.csv'
test_path = 'test.csv'
readme = 'readme.txt'

Take only part of it, cos google doesn't have enough RAM for us

In [ ]:
data = pd.read_csv(path/train_path, nrows=150000)
data.head()

,5,"dr. goldberg offers everything i look for in a general practitioner. he's nice and easy to talk to without being patronizing; he's always on time in seeing his patients; he's affiliated with a top-notch hospital (nyu) which my parents have explained to me is very important in case something happens and you need surgery; and you can get referrals to see specialists without having to see him first. really, what more do you need? i'm sitting here trying to think of any complaints i have about him, but i'm really drawing a blank."
0,2,"Unfortunately, the frustration of being Dr. Goldberg's patient is a repeat of the experience I've had with so many other doctors in NYC -- good doctor, terrible staff. It seems that his staff simply never answers the phone. It usually takes 2 hours of repeated calling to get an answer. Who has time for that or wants to deal with it? I have run into this problem with many other doctors and I just don't get it. You have office workers, you have patients with medical needs, why isn't anyone answering the phone? It's incomprehensible and not work the aggravation. It's with regret that I..."
1,4,"Been going to Dr. Goldberg for over 10 years. I think I was one of his 1st patients when he started at MHMG. He's been great over the years and is really all about the big picture. It is because of him, not my now former gyn Dr. Markoff, that I found out I have fibroids. He explores all options with you and is very patient and understanding. He doesn't judge and asks all the right questions. Very thorough and wants to be kept in the loop on every aspect of your medical health and your life."
2,4,Got a letter in the mail last week that said Dr. Goldberg is moving to Arizona to take a new position there in June. He will be missed very much. \n\nI think finding a new doctor in NYC that you actually like might almost be as awful as trying to find a date!
3,1,"I don't know what Dr. Goldberg was like before moving to Arizona, but let me tell you, STAY AWAY from this doctor and this office. I was going to Dr. Johnson before he left and Goldberg took over when Johnson left. He is not a caring doctor. He is only interested in the co-pay and having you come in for medication refills every month. He will not give refills and could less about patients's financial situations. Trying to get your 90 days mail away pharmacy prescriptions through this guy is a joke. And to make matters even worse, his office staff is incompetent. 90% of the time when you c..."
4,5,Top notch doctor in a top notch practice. Can't say I am surprised when I was referred to him by another doctor who I think is wonderful and because he went to one of the best medical schools in the country. \nIt is really easy to get an appointment. There is minimal wait to be seen and his bedside manner is great.


Size is:

In [ ]:
data.size

300000

We need to know column names

In [ ]:
data.columns.tolist()

['5',
 "dr. goldberg offers everything i look for in a general practitioner.  he's nice and easy to talk to without being patronizing; he's always on time in seeing his patients; he's affiliated with a top-notch hospital (nyu) which my parents have explained to me is very important in case something happens and you need surgery; and you can get referrals to see specialists without having to see him first.  really, what more do you need?  i'm sitting here trying to think of any complaints i have about him, but i'm really drawing a blank."]

Column name with texts, looks strange, but we use it, cos there is texts

In [ ]:
label_column, text_column = data.columns.tolist()

In [ ]:
dls = TextDataLoaders.from_df(
    df=data,
    text_col=text_column,
    label_col=label_column
)

Save it, to next time, just load it, instead of tokenize again.

Doesn't work first method, used from torch

In [ ]:
# dls.save('default.pkl')
# dls = load_data(path)

torch.save(dls, 'default.pkl')

Check on what device are we does it (is CPU are TPU?)

In [ ]:
dls.device

device(type='cuda', index=0)

In [ ]:
dls.show_batch()

,text,category
0,"xxbos a xxup note xxup to xxup the xxup owner : \n xxmaj your employee , the scrawny , balding man , should be fired . \n xxmaj he was yelling at me and saying extremely rude things instead of fixing the nails to make the customer happy . xxmaj i 've never been treated this way by any company that xxmaj i 've ever done business with . i urge you to take a few moments to read about what happened from the customer 's point of view , as xxmaj i 'm sure that you may hear about it but from a different light that is favorable to the scrawny , balding man . \n\n xxup my xxup review : xxmaj they should n't even get xxup one xxup star . xxmaj the scrawny man with the receding hair that works there is the ultimate cause of this",1
1,"xxbos xxmaj i 've been coming to this place since xxmaj i 've moved to xxup az about 10 years ago . i ate here so often that at one point i was a "" mayor "" on foursquare and "" duchess "" on xxmaj yelp ! xxmaj they 've changed ownership twice since 10 years ago , and the new one took over about 4 - 5 years ago i believe . xxmaj their food 's ok , but the best thing on the menu is their kimchi jjigae . xxmaj that 's the reason why i came here all those times and i do n't even touch their banchan , because it 's so subpar compared to xxup la , xxup sf , or xxup nyc that it 's laughable . xxmaj their xxmaj soon xxmaj tofu is average and the xxmaj kalbi is average as well .",1
2,"xxbos xxmaj wanted : atmosphere , good wine and decent food for xxmaj sunday xxunk dinner . \n xxmaj received : expensive place , mediocre wine and salty , processed food in a place that looked like it had seen better days . \n\n i will start out and say we had the nicest staff ! xxmaj no complaints at all on our service ! i feel bad that they have to serve this food in this place and try to do it with a smile . i do n't think i could ( and i have worked food for years ) \n\n xxmaj we called on the off chance xxmaj chianti 's would be open on a xxmaj xxunk and they were ( though they will be closing xxmaj sun and xxmaj mon for "" the season "" starting next week . i did n't realize there was a",2
3,"xxbos xxmaj nope … \n\n xxmaj not going to give anything more than 1 star . xxmaj it 's the challenge of giving no review vs. giving 1 star . \n\n i returned to this xxmaj thai xxmaj basil . xxmaj while they have managed to clean up the dust and dirt i saw previously , the wait , the poor presentation of the food i ordered and the lack of any feeling of care from the staff have me not returning to this xxmaj thai xxmaj basil location . \n\n i waited at the entrance in front of the ' please wait to be seated ' sign for 5 minutes . xxmaj at 4 xxup pm , there were 3 tables seated . 2 were well into their meals and one appeared to be ready to place their order . xxmaj the hostess who finally came to seat me",1
4,"xxbos xxmaj i 'm in the market for a good place to become a regular so i tried here its close to where i work so i tried . i went in for a pedi and a gel mani and a brow wax , with my friend who also wanted all the same services besides the wax we were both seated at the same time starting with our pedis . \n\n xxmaj my experience - the xxmaj woman who did my pedi was great talked , and friendly she was great . i stand all day and it showed on my feet , she removed every callus and did everything that went with the ultimate pedi polished my toes great ! xxmaj then the ladies switched my lady went to do my friends gel mani and i went to do my brows with the lady that had just worked on",1
5,"xxbos i have to preface this review with two factors . xxmaj one , i moved here from xxmaj south xxmaj florida , where the best burgers in town came from your average diners . xxmaj they were the last bastion of xxmaj american food in xxmaj south xxmaj florida , so they served up xxup real xxmaj american food . xxmaj if you wanted a real burger , you went to a diner . xxmaj after returning to xxmaj pittsburgh , i found that every

accuracy - Вычисление точности с помощью targ, когда pred равно bs * n_classes

Perplexity - экспонента потери перекрестной энтропии

In [ ]:
learn = text_classifier_learner(
    dls=dls,
    arch=AWD_LSTM,
    drop_mult=0.5,
    metrics=accuracy
)

In [ ]:
learn.fine_tune(4, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,1.251443,1.143428,0.499367,06:25


epoch,train_loss,valid_loss,accuracy,time
0,0.943040,0.920920,0.594967,16:06
1,0.884007,0.865785,0.614200,16:06


epoch,train_loss,valid_loss,accuracy,time
0,0.943040,0.920920,0.594967,16:06
1,0.884007,0.865785,0.614200,16:06
2,0.804150,0.790552,0.655333,16:05
3,0.738590,0.797883,0.656033,16:07


In [ ]:
learn.save('learn_on_accuracy')

Path('models/learn_on_accuracy.pth')

In [ ]:
learn.show_results()

,text,category,category_
0,"xxbos xxmaj wow , this place is a xxup complete xxup scam & please do n't waste your time here . xxmaj talk about xxup substandard xxup service & a xxup business xxup who xxup only xxup cares xxup about xxup collecting xxup money ! i rarely even leave reviews but my experience was xxup so xxup awful & xxup unprofessional with such xxup horrible & xxup unprofessional service i felt i had to let others know not to waste their money here .. \n\n▁ xxmaj the owner claims to have years of experience & done thousands of resumes , but all your time in the beginning in spent with her assistant , as she simply introduced herself & disappeared for the entire 2 hours that my initial appt ended up running . \n\n▁ xxmaj they insist you need a more fancy resume package once they have you in their",1,1
1,"xxbos xxmaj beware , it 's cheap for a reason . xxmaj at it 's best , it mediocre , at it 's worst , it 's complete garbage . xxmaj i 've been here several times , but this will be the last . \n\n xxmaj let 's start with check - in . xxmaj they never have enough people at the check in desk , so enjoy a long wait . xxmaj also , if you booked a king bed , be prepared to fight for it as they never have any . xxmaj they pull this : \n\n xxmaj clerk : xxmaj you wanted non - smoking ? \n xxmaj me : xxmaj yes \n xxmaj clerk : xxmaj we have no more kings in non - smoking . \n xxmaj me : xxmaj fine , smoking then . \n xxmaj clerk : xxmaj oh , we",2,1
2,"xxbos xxmaj it 's in a real crappy part of town . xxmaj prostitutes and xxmaj drug dealers galore , only thing that breaks the scenery is stepping over the homeless ! xxmaj so , the location may not be something that xxmaj harrie 's can control , but their service xxup is . xxmaj my husband , visited the store earlier this morning to get our family breakfast . i am newly pregnant and was craving a lox and cream cheese bagel with the traditional tomato and lettuce . xxmaj he arrives home with a lox cream cheese xxup spread ( not even a slice of lox … they say they are like a real xxup ny xxmaj bagel xxmaj place , having been to xxup ny several times , and having loved ones their , i xxup know that the spread is the kiss of failure in a",1,1
3,"xxbos xxmaj oh my dear lord . xxmaj this place man . xxmaj when i think of xxmaj vegas i think of a xxmaj disneyland for adults . i mean there are people who are out on the strip night and day handing out cards with naked women on them . xxmaj this place is called xxmaj sin xxmaj city for a reason and with all the places you can take your kids on vacation in the world i would suggest avoiding xxmaj las xxmaj vegas . xxmaj well most adults ( with a brain in their head ) feel that way but i have some sad news folks , according to xxmaj mary the xxmaj manager of xxmaj guest xxmaj services at xxmaj the xxmaj excalibur they only care about catering to families with kids and making concessions and accommodations for them . xxmaj if you are apart of",2,2
4,"xxbos xxmaj this one is really hard . \n\n xxmaj based on other xxmaj yelp reviews , i went to xxmaj eliana 's to pick up lunch last week . xxmaj it 's close to my work , and it sounded like such a nice departure from my usual lunchtime rounds . xxmaj this past weekend , my boyfriend and i went to dinner there , because i thought it deserved a second try . xxmaj i 'll tell you a little about both experiences . \n\n xxmaj after perusing all of the ( mostly ) helpful and honest reviews here , i really psyched myself out about giving it a try and just xxup had to go there for takeout that xxup very xxup same xxup day - xxmaj i 'm not a delayed gratification kind of person , despite my best efforts . i ended up at the",2,3
5,"xxbos xxup updated : xxmaj just wanted to share that we attempted to visit for a third time when my husband 's brother was in town in mid - november of 2014 . xxmaj we had reservations for 7:30 on a xxmaj tuesday , so we obviously figured we would be seated immediately . 

Now we can predict new texts



In [ ]:
learn.predict("I really liked doctor")

('4', tensor(3), tensor([0.0191, 0.0302, 0.0922, 0.5177, 0.3408]))

In [ ]:
learn.predict("When i was on doctor, it was really weird how he talking with me, I think next time i visit other doctor")

('3', tensor(2), tensor([0.1000, 0.2371, 0.4054, 0.1847, 0.0727]))

In [ ]:
learn.predict("It was the worst visit")

('1',
 tensor(0),
 tensor([9.6215e-01, 3.5394e-02, 1.4914e-03, 1.8891e-04, 7.7889e-04]))